<a href="https://colab.research.google.com/github/uomodellamansarda/colab_practice_and_tutorial/blob/master/GoogleTrends3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Section 1
# Modules Importing

We are comparing different trend about different cities on gtrend normalizing data

In [0]:
#Section:     1
#Subsection:  A

#Module importing
import pandas as pd 
import numpy as np 
import gspread
import os
import glob
import matplotlib.pyplot as plt

from datetime import timedelta

In [0]:
#Section:      1
#Subsection:   B
#Importing the modules for clustering 
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import AffinityPropagation
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

In [0]:
#Section:     1
#Subsection:  C
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gs = gspread.authorize(gauth.credentials)

Section 2
# Manipulating Data From Google Sheet

In [0]:
#Section:     2
#Subsection:  D

#creating a converting function 
#that allow us to modify <1
#to a number
#just for the example I replaced <1 with 0.5
#we will use also to cast from object to numbers
#the values we are going to read from the google sheet
def df_converter(df):
  df=df.iloc[3:].reset_index(drop=True).copy()

  less_one='<1'
  #first column --> time stamp 
  header=df.columns
  df[header[0]]=pd.to_datetime(df[header[0]])
  df=df.replace('<1',0.5)
  # converting to float 
  df[header[1:]]=df[header[1:]].apply(pd.to_numeric)
  df_converted=df.copy()
  return df_converted

In [0]:
#Section:     2
#Subsection:  E
#from here we are going to read the values from our excel file

namepath="https://docs.google.com/spreadsheets/d/1CEsfoCOcV8EuVCRx6rhJBSFs-cSWaF-ksMLryiw5jEo/edit?usp=sharing"
#we create a gs instance passing the google sheet url 
#as input
sht2 = gs.open_by_url(namepath)

In [6]:
#Section:     2
#Subsection:  F
#we read the first tab from the google sheet 
worksheet_nord = sht2.get_worksheet(0)
#we read the second tab from the google sheet 
sud_sheet=sht2.get_worksheet(1)
#we get all the values
sud_data=sud_sheet.get_all_values()
#we identify (manually I am sorry)
#where the header is
sud_headers=sud_sheet.row_values(3)
#we create a first row DF
df_sud=pd.DataFrame(sud_data, columns=sud_headers)
print(type(worksheet_nord))
headers_nord=worksheet_nord.row_values(3)
data_nord =worksheet_nord.get_all_values()
df_nord=pd.DataFrame(data_nord,columns=headers_nord)

<class 'gspread.models.Worksheet'>


In [0]:
#Section:     2
#Subsection:  G
#applying the converting function
#to our dataframe
df_sud_converted=df_converter(df_sud)
df_nord_converted=df_converter(df_nord)

In [8]:
#Section:     2
#Subsection:  H
#inspecting the df 
#that we created and converted
print(df_sud_converted.head())
print(df_nord_converted.head())
print(df_sud_converted.info())
print(df_nord_converted.info())

        Week  Bari: (Italy)  ...  Reggio Calabria: (Italy)  Palermo: (Italy)
0 2020-01-05             14  ...                         4                18
1 2020-01-12             13  ...                         4                18
2 2020-01-19             13  ...                         4                17
3 2020-01-26             13  ...                         5                17
4 2020-02-02             13  ...                         4                17

[5 rows x 6 columns]
        Week  Milan: (Italy)  ...  Venice: (Italy)  Florence: (Italy)
0 2020-01-05              83  ...               11                1.0
1 2020-01-12              86  ...               11                1.0
2 2020-01-19              82  ...               11                1.0
3 2020-01-26              82  ...               11                1.0
4 2020-02-02              81  ...               12                1.0

[5 rows x 6 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data

Section 3
#Plotting data with bokeh

In [0]:
#Section:     3
#Subsection:  I
# we now create a plot to viz all the data
# Standard imports 

from bokeh.io import output_notebook, show
output_notebook()

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure



In [0]:
#Section:     3
#Subsection:  L
#honestly I could use the datetime datatype to plot 
#but I had to fix some minor issues and I prefered to act this way
#not the best way, I think is better to deal with datetime datatype
#We should convert to datetime column datasource
df_sud_converted['Week']=df_sud_converted['Week'].dt.strftime('%Y-%m-%d')


In [11]:
#Section:     3
#Subsection:  M
#creating the ColumnDataSource
#for the first plot 
source = ColumnDataSource(df_sud_converted)
#creating and array of numbers
#we will use for the xaxis ticker
x_axis=np.arange(0,len(df_sud_converted['Week']))
source.add(x_axis,'x_axis')
#just exploring 
#our ColumnDataSource
print(source.data)

{'index': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]), 'Week': array(['2020-01-05', '2020-01-12', '2020-01-19', '2020-01-26',
       '2020-02-02', '2020-02-09', '2020-02-16', '2020-02-23',
       '2020-03-01', '2020-03-08', '2020-03-15', '2020-03-22',
       '2020-03-29', '2020-04-05', '2020-04-12', '2020-04-19'],
      dtype=object), 'Bari: (Italy)': array([14, 13, 13, 13, 13, 13, 14, 13, 13, 10,  8,  7,  7,  6,  7,  7]), 'Rome: (Italy)': array([ 90, 100,  91, 100,  88,  76,  76,  74,  75,  48,  39,  37,  45,
        44,  46,  49]), 'Naples: (Italy)': array([0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. , 1. , 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5]), 'Reggio Calabria: (Italy)': array([4, 4, 4, 5, 4, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3]), 'Palermo: (Italy)': array([18, 18, 17, 17, 17, 16, 17, 20, 17, 14, 11, 11, 11, 10, 12, 11]), 'x_axis': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])}


In [12]:
#Section:     3
#Subsection:  N
l = figure(title="South Trend", plot_width=500, plot_height=300)

glyph_1 = l.line('Week','Palermo: (Italy)',source=source,  color='red',
                 legend_label="Palermo")
l.line('x_axis','Bari: (Italy)',source=source,  color='blue',
       legend_label="Bari")
l.line('x_axis','Reggio Calabria: (Italy)',source=source,  color='blue',
       legend_label="Reggio Calabria")
l.line('x_axis','Naples: (Italy)',source=source,  color='gold',
       legend_label="Naples")
l.line('x_axis','Rome: (Italy)',source=source,  color='green',
       legend_label="Rome")
l.xaxis.ticker=x_axis
#
xaxis_labels_dict_sud=dict(zip(x_axis.tolist(), df_sud_converted['Week']))
l.xaxis.major_label_overrides=xaxis_labels_dict_sud
l.xaxis.major_label_orientation = "vertical"

show(l)

In [13]:
#Section:     3
#Subsection:  N
#We should convert to datetime column datasource
df_nord_converted['Week']=df_nord_converted['Week'].dt.strftime('%Y-%m-%d')

source_nord = ColumnDataSource(df_nord_converted)

x_axis=np.arange(0,len(df_nord_converted['Week']))
source_nord.add(x_axis,'x_axis')

'x_axis'

In [14]:
#Section:     3
#Subsection:  P
l_second = figure(title="Nord Trend", plot_width=500, plot_height=300)

glyph_2 = l_second.line('Week','Florence: (Italy)',source=source_nord,  color='red',
                 legend_label="Florence")
l_second.line('x_axis','Milan: (Italy)',source=source_nord,  color='blue',
       legend_label="Milan")
l_second.line('x_axis','Rome: (Italy)',source=source_nord,  color='blue',
       legend_label="Rome")
l_second.line('x_axis','Turin: (Italy)',source=source_nord,  color='gold',
       legend_label="Turin")
l_second.line('x_axis','Venice: (Italy)',source=source_nord,  color='green',
       legend_label="Venice")
l_second.xaxis.ticker=x_axis
#
xaxis_labels_dict_nord=dict(zip(x_axis.tolist(), df_nord_converted['Week']))
l_second.xaxis.major_label_overrides=xaxis_labels_dict_nord

l_second.xaxis.major_label_orientation = "vertical"
show(l_second)


In [15]:
#Section:     3
#Subsection:  Q
#trying to create a dictonary with time and x_axis

zip_xaxis=zip(x_axis,df_nord_converted['Week'])
dict_xaxis=dict(zip_xaxis)
print(dict_xaxis)

{0: '2020-01-05', 1: '2020-01-12', 2: '2020-01-19', 3: '2020-01-26', 4: '2020-02-02', 5: '2020-02-09', 6: '2020-02-16', 7: '2020-02-23', 8: '2020-03-01', 9: '2020-03-08', 10: '2020-03-15', 11: '2020-03-22', 12: '2020-03-29', 13: '2020-04-05', 14: '2020-04-12', 15: '2020-04-19'}


In [16]:
#Section:     3
#Subsection:  R
# take the second shee sud and apply the function
#df_sud_casted=df_converter(df_sud)
print(df_sud.info())
print(df_sud.columns[0])
print(df_sud.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Week                      19 non-null     object
 1   Bari: (Italy)             19 non-null     object
 2   Rome: (Italy)             19 non-null     object
 3   Naples: (Italy)           19 non-null     object
 4   Reggio Calabria: (Italy)  19 non-null     object
 5   Palermo: (Italy)          19 non-null     object
dtypes: object(6)
memory usage: 1.0+ KB
None
Week
                       Week  ...  Palermo: (Italy)
0  Category: All categories  ...                  
1                            ...                  
2                      Week  ...  Palermo: (Italy)
3                2020-01-05  ...                18
4                2020-01-12  ...                18

[5 rows x 6 columns]
